## GHG Emissions
The basis for these emission factors is the Handbook of Emission
Factors for Road Transport (HBEFA Version 3.3). (Notter et al.,2019).(3)
$$ E = A * f * l * e_f $$
Where:

E is the total CO2 emissions (gram CO2 per day)

A is the average number of vehicles of type A per day

F is the specific fuel consumption per vehicle type A (litres/km)

l is the length of the road (km)

$e_f$ is the emission factor (grams CO2/litre)

## Imports

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import torch
import random
import cv2
from tqdm import tqdm
from matplotlib import pyplot as plt
import segmentation_models_pytorch as smp
import albumentations as album
from PIL import Image
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import random_split
from sklearn.metrics import accuracy_score
import plotly.express as px
import torchmetrics
from torchmetrics import MeanAbsolutePercentageError


%matplotlib inline

/home/ah2719/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Global Variables

In [8]:
AADT_PATH = "/home/ah2719/FYP/Spatial_Finance_Transport/ground_truth_data/AADT.csv"
IMAGE_METADATA_PATH = "/home/ah2719/FYP/Spatial_Finance_Transport/data/image_metadata.csv" # Useful data: vehicle_count, vehicle_bb, road_segment_length, speed_estimate
EMISSIONS_FACTORS_PATH = "/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/uk_emissions_factors.csv" # emission factor (grams CO2/litre)
SPECIFIC_FUEL_CONSUMPTION_PATH = "/home/ah2719/FYP/Spatial_Finance_Transport/ground_truth_data/fuel_consumption_per_vehicle.csv" # specific fuel consumption per vehicle type A (litres/km)

## Generic Functions

## AADT to GHG Emissions
Multiply AADT for vehicle types by length of road segment

### Load Data

In [ ]:
df_aadt = pd.read_csv(AADT_PATH)
aadt = df_aadt['aadt']
df_aadt.head()

In [ ]:
# Each row should represent a vehicle "type"
df_image_metadata = pd.read_csv(IMAGE_METADATA_PATH)

vehicle_bb_sizes = df_image_metadata['vehicle_bb_size'] 
vehicle_counts = df_image_metadata['vehicle_count']
road_segment_length = df_image_metadata['road_segment_length']
speed_estimation = df_image_metadata['speed_estimate']

df_image_metadata.head()

In [13]:
df_emissions_factors = pd.read_csv(EMISSIONS_FACTORS_PATH)
df_emissions_factors['total'] = df_emissions_factors.sum(axis=1)
df_emissions_factors.head() # g/km

/tmp/ipykernel_7025/212430925.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_emissions_factors['total'] = df_emissions_factors.sum(axis=1)


,Unnamed: 0,NOx,PM10,PM2.5,CO,VOC,NH3,SO2,Benzene,N2O,total
0,Petrol cars,0.065,0.001,0.001,0.583,0.090,0.013,0.000,0.002,0.001,0.756
1,Diesel cars,0.517,0.008,0.008,0.047,0.004,0.004,0.001,0.000,0.006,0.595
2,Petrol LGVs,0.094,0.001,0.001,1.234,0.085,0.017,0.001,0.002,0.002,1.437
3,Diesel LGVs,0.808,0.007,0.007,0.058,0.008,0.005,0.001,0.000,0.006,0.900
4,Rigid HGVs,1.428,0.020,0.020,0.438,0.039,0.009,0.002,0.000,0.031,1.987


In [1]:
#df_specifc_fuel_consumption = pd.read_csv(SPECIFIC_FUEL_CONSUMPTION_PATH)
#df_specifc_fuel_consumption.head()

NameError: name 'pd' is not defined

## Method

### TODO: Convert vehicle BB co-ordinates to BB areas

In [ ]:
# df_image_metadata['vehicle_area'] = length * width of vehicle bb's

### TODO: Convert vehicle BB area to vehicle type

In [ ]:
# 0 = passenger car, 1 = small truck, 2 = HGV etc
# df_image_metadata['vehicle_type'] = 0 if df_image_metadata['vehicle_bb_size'] < 500, etc.
# df_image_metadata['count'] = df_image_metadata.

### TODO: Set emissions factors from vehicle type

In [ ]:
# df_image_metadata['vehicle_type_emissions_factor']  = df_emissions_vehicle_type

### TODO: Apply portions of aadt estimate to each vehicle

In [ ]:
reciprocal_no_of_vehicles = 1 / len(df_image_metadata)
df_image_metadata['aadt_vehicles'] = aadt * reciprocal_no_of_vehicles

### TODO: GHG Emissions calculation

In [ ]:
aadt_emissions = 0

for i in range(len(df_image_metadata)):
    aadt_emissions += road_segment_length * df_image_metadata.iloc[i]['aadt_vehicles'] * df_image_metadata.iloc[i]['vehicle_type_emissions_factor']

aadt_emissions